# Crawler

This notebook contains started code structure for creating a crawler on single machine

**Author:** Noshaba Nasir

**Date:** 26/3/2021

**Updated by:** <Write your name and Roll number here>
    

In [1]:
import os 
import random
from tld import get_tld
from advertools import robotstxt_test
from urllib.parse import urlsplit, urlunsplit
from bs4 import BeautifulSoup
import requests
import requests.exceptions
import threading
import datetime
import time
# Add any library to be imported here

In [2]:
# Crawler Parameters
BACKQUEUES= 3
THREADS= BACKQUEUES*3
FRONTQUEUES= 5
WAITTIME= 15 ; # wait 15 seconds before fetching URLS from 
front = None
Links=[]
# Add any other global parameters here

# FRONTIER

Frontier should use the Mercator frontier design as discussed in lecture.

Preferably it should be a class and should have the given functions.

*prioritizer* function is a stub right now, it will return a random number  between 1 to f for given URL 

In [3]:
def getTime(obj):
    return obj["time"]

In [4]:
class frontier:
# add the code for frontier here
# should have functions __init__, get_URL, add_URLs, add_to_backqueue
    def __init__(self):
        '''
        Initializer
        '''
        self.front = []
        self.back = []
        for i in range(FRONTQUEUES):
            self.front.append([])
        for i in range(BACKQUEUES):
            self.back.append({"time":datetime.datetime.now(),"list":[]})
        self.seed= ["https://docs.oracle.com/en/","https://www.oracle.com/corporate/","https://en.wikipedia.org/wiki/Machine_learning","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html","https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html","https://en.wikipedia.org/w/api.php","https://en.wikipedia.org/api/","https://en.wikipedia.org/wiki/Weka_(machine_learning)"]
        for link in self.seed:
            self.front[prioritizer(link,FRONTQUEUES)-1].append(link)
        None # replace this with your code
    def add_URL(self,url):
        self.front[prioritizer(url,FRONTQUEUES)-1].append(url)
    def add_to_backqueue(self):
        selected = random.randint(1,FRONTQUEUES)-1
        if len(self.front[selected]) > 0:
            link=self.front[selected].pop(0)
            domain = get_tld(link, as_object=True).fld
            des = []
            des2 = []
            for i in range(BACKQUEUES):
                if len(self.back[i]["list"]) == 0:
                    des2.append(i)
                elif domain in self.back[i]["list"][0]:
                    des.append(i)
            if des == [] and len(des2) >= 1:
                self.back[des2[0]]["list"].append(link)
            elif des2 == [] and len(des) >=1:
                self.back[des[0]]["list"].append(link)
            else:
                self.front[selected].append(link)
    def get_URL(self):
        self.back.sort(key=getTime)
        if self.back[0]["time"] > datetime.datetime.now():
            patience=self.back[0]["time"] - datetime.datetime.now()
            if patience.seconds > 0:
                time.sleep(patience.seconds)
                print("Sleeping for "+str(patience.seconds)+"\n")
        if len(self.back[0]["list"]) > 0:
            self.back[0]["time"]=datetime.timedelta(seconds=WAITTIME)+datetime.datetime.now()
            return self.back[0]["list"].pop(0)
        return None
        

def prioritizer(URL,f):
    """
    Take URL and returns priority from 1 to F
    Right now it like a stub function. 
    It will return a random number from 1 to f for given inputs. 
    """
    return random.randint(1,f)
    

# FILTER URLS

Filter the URLS that are in robots.txt files of server and the have been already processed. 

In [5]:
# write code for filtering urls here 
def filter_url(url):
    try:
        parts = urlsplit(url)
        base = "{0.scheme}://{0.netloc}".format(parts)
        return not robotstxt_test(base+'/robots.txt', user_agents=['*'], urls=[url])["can_fetch"].values[0]
    except:
        return False

## ---------------------Write any other codes/data require to run the crawler above this cell-----------

# Run Crawler

In [6]:
# Theard task
def crawler_thread_task():
    global front
    url = front.get_URL()
    if url:
        Links.append(url)
        front.add_to_backqueue()
        print(" reached "+url+"\n")
        req = requests.get(url)
        res = BeautifulSoup(req.content, 'html.parser')
        urls = res.find_all('a',href=True)
        urls=[a['href'] for a in urls if 'http' in a['href'] or a['href'][0] == "/"]
        parts = urlsplit(url)
        base = "{0.scheme}://{0.netloc}".format(parts)      
        urls=[a  if 'http' in a else base+a for a in urls]
        urls=[a for a in urls if not filter_url(a)]
        for url in urls:
            try:
                r = requests.head(url)
                if "text/html" in r.headers["content-type"]:
                    front.add_URL(url)
            except:
                front.add_to_backqueue()
                pass
    front.add_to_backqueue()
    None # define individual crawler thread's function here as studies in class

In [7]:
# intialize every thing
front = frontier()
while len(Links) < 1001:
    for i in range(THREADS):
        thread = threading.Thread(target=crawler_thread_task)
        thread.start()
    time.sleep(WAITTIME)

 reached https://www.oracle.com/corporate/

 reached https://en.wikipedia.org/w/api.php

 reached https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html

Sleeping for 14
Sleeping for 14
Sleeping for 14

 reached https://docs.oracle.com/en/

Sleeping for 14
Sleeping for 14
Sleeping for 14





 reached https://en.wikipedia.org/wiki/Weka_(machine_learning)

 reached https://en.wikipedia.org/wiki/Machine_learning
 reached https://www.csie.ntu.edu.tw/~cjlin/libsvm/index.html


Sleeping for 14
Sleeping for 14
Sleeping for 14
Sleeping for 14
Sleeping for 14





 reached https://www.oracle.com/corporate/accessibility/

 reached https://en.wikipedia.org/api/

 reached https://foundation.wikimedia.org/wiki/Developer_app_guidelines

 reached https://www.oracle.com/universal-menu/



Exception in thread Thread-99:
Traceback (most recent call last):
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\connectionpool.py", line 382, in _make_request
    self._validate_conn(conn)
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\connectionpool.py", line 1010, in _validate_conn
    conn.connect()
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\connection.py", line 421, in connect
    tls_in_tls=tls_in_tls,
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\util\ssl_.py", line 429, in ssl_wrap_socket
    sock, context, tls_in_tls, server_hostname=server_hostname
  File "c:\users\zarar\appdata\local\programs\python\python37-32\lib\site-packages\urllib3\util\ssl_.

KeyboardInterrupt: 

## ------------------------------------------------------End of Notebook---------------------------------------------------